In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

DATADIR = "F:\PetImages"

CATEGORIES = ["Dog1", "Cat1"]

for category in CATEGORIES:  # do dogs and cats
    path = os.path.join(DATADIR,category)  # create path to dogs and cats
    for img in os.listdir(path):  # iterate over each image per dogs and cats
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
        IMG_SIZE = 100
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        plt.imshow(new_array, cmap='gray')  # graph it
        plt.show()  # display!

        break  # we just want one for now so break
    break  #...and one more!
#print(img_array)
#print(img_array.shape)
#IMG_SIZE = 150
#new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
#plt.imshow(new_array, cmap='gray')
#plt.show()


<Figure size 640x480 with 1 Axes>

In [2]:
training_data = []

def create_training_data():
    for category in CATEGORIES:  # do dogs and cats

        path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat

        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
            #except OSError as e:
            #    print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #    print("general exception", e, os.path.join(path,img))

create_training_data()

print(len(training_data))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 486/486 [00:00<00:00, 588.53it/s]


971


In [3]:
import random

random.shuffle(training_data)
for sample in training_data[:10]:
    print(sample[1])

1
1
1
0
1
0
0
1
1
0


In [4]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

[[[[ 76]
   [ 78]
   [ 76]
   ...
   [224]
   [223]
   [225]]

  [[ 64]
   [ 62]
   [ 68]
   ...
   [223]
   [223]
   [221]]

  [[ 58]
   [ 67]
   [ 80]
   ...
   [216]
   [223]
   [222]]

  ...

  [[ 73]
   [ 69]
   [ 57]
   ...
   [241]
   [241]
   [241]]

  [[ 90]
   [ 65]
   [ 51]
   ...
   [241]
   [241]
   [241]]

  [[ 60]
   [ 69]
   [ 69]
   ...
   [241]
   [241]
   [241]]]]


In [5]:
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()
#We can always load it in to our current script, or a totally new one by doing:

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=10,
                      validation_split=0.3,
                      callbacks=[tensorboard])

model.save('64x3-CNN.model')

3-conv-64-nodes-0-dense-1549450564
Train on 679 samples, validate on 292 samples
Epoch 1/10
679/679 [==============================] - ETA: 24s - loss: 0.6910 - acc: 0.53 - ETA: 20s - loss: 0.6967 - acc: 0.51 - ETA: 18s - loss: 0.6944 - acc: 0.53 - ETA: 17s - loss: 0.6906 - acc: 0.55 - ETA: 16s - loss: 0.6915 - acc: 0.53 - ETA: 15s - loss: 0.6925 - acc: 0.52 - ETA: 14s - loss: 0.6943 - acc: 0.50 - ETA: 13s - loss: 0.6938 - acc: 0.50 - ETA: 12s - loss: 0.6941 - acc: 0.50 - ETA: 11s - loss: 0.6937 - acc: 0.51 - ETA: 10s - loss: 0.6945 - acc: 0.50 - ETA: 9s - loss: 0.6941 - acc: 0.5078 - ETA: 8s - loss: 0.6937 - acc: 0.509 - ETA: 7s - loss: 0.6941 - acc: 0.506 - ETA: 6s - loss: 0.6948 - acc: 0.500 - ETA: 5s - loss: 0.6946 - acc: 0.502 - ETA: 4s - loss: 0.6944 - acc: 0.509 - ETA: 3s - loss: 0.6942 - acc: 0.512 - ETA: 2s - loss: 0.6942 - acc: 0.509 - ETA: 1s - loss: 0.6941 - acc: 0.506 - ETA: 0s - loss: 0.6945 - acc: 0.495 - 25s 36ms/step - loss: 0.6945 - acc: 0.4963 - val_loss: 0.6924 - va

In [4]:
import cv2
import tensorflow as tf

CATEGORIES = ["Dog0", "Cat0"]


def prepare(filepath):
    IMG_SIZE = 100  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)


model = tf.keras.models.load_model("64x3-CNN.model")

prediction = model.predict([prepare('9909.jpg')])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])

[[1.]]
Cat0
